In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import numpy as np
import json

## Map countries to remove duplication

Countries must be mapped as there are multiple representations of each country in the dataset.

In [49]:
# Read the country mapping from the JSON file
with open("country_mapping.json", "r") as json_file:
    country_mapping = json.load(json_file)

In [50]:
# Read the CSV file into a DataFrame
login_dataframe = pd.read_csv("Dataset/login.csv")

# Remap the 'country' column using the country_mapping dictionary
login_dataframe['country'] = login_dataframe['country'].map(country_mapping).fillna(login_dataframe['country'])
login_dataframe['reg_date'] = pd.to_datetime(login_dataframe['reg_date'], unit='s')

login_dataframe.head()

,login,country,account_currency,reg_date
0,457547,Romania,EUR,2021-02-25 00:15:32
1,474589,Canada,CAD,2021-01-07 02:44:02
2,504321,Canada,CAD,2020-10-14 02:31:50
3,504322,Canada,USD,2020-10-15 04:35:45
4,504326,Canada,USD,2020-10-19 07:39:12


In [51]:
login_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40512 entries, 0 to 40511
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   login             40512 non-null  int64         
 1   country           40505 non-null  object        
 2   account_currency  40512 non-null  object        
 3   reg_date          40512 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.2+ MB


In [52]:
daily_reports_df = pd.read_csv("Dataset/daily_report.csv")
daily_reports_df['record_time'] = pd.to_datetime(daily_reports_df['record_time'])

## Load trades dataframe and merge with login

Only accounts that have registered within the timeframe of trades.csv are considered. This is because we do not have complete trading data for accounts before that and do not want to misrepresent data which may negatively influence a predictive model.

In [53]:
# Load and prepare the trades data frame
trades_dataframe = pd.read_csv("Dataset/trades.csv")
trades_dataframe['open_time'] = pd.to_datetime(trades_dataframe['open_time'], unit='s')
trades_dataframe['close_time'] = pd.to_datetime(trades_dataframe['close_time'], unit='s')


In [54]:
# Range of trades_dataframe
min_trade_date1 = trades_dataframe['open_time'].min()
max_trade_date1 = trades_dataframe['close_time'].max()
print(max_trade_date1 - min_trade_date1)

422 days 23:55:32


In [55]:
trades_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521777 entries, 0 to 4521776
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ticket       int64         
 1   login        int64         
 2   symbol       object        
 3   cmd          int64         
 4   volume       float64       
 5   open_time    datetime64[ns]
 6   open_price   float64       
 7   close_time   datetime64[ns]
 8   close_price  float64       
 9   tp           float64       
 10  sl           float64       
 11  reason       int64         
 12  commission   float64       
 13  swaps        float64       
 14  profit       float64       
 15  volume_usd   float64       
dtypes: datetime64[ns](2), float64(9), int64(4), object(1)
memory usage: 552.0+ MB


In [56]:
# Check no. unique accounts in trades
trades_dataframe['login'].nunique()

11976

In [57]:
# Identify the minimum and maximum trade dates
min_trade_date = trades_dataframe['open_time'].min()
max_trade_date = trades_dataframe['close_time'].max()

# Filter the login dataframe to include only accounts registered within the trade dates range
filtered_login_dataframe = login_dataframe[(login_dataframe['reg_date'] >= min_trade_date) & (login_dataframe['reg_date'] <= max_trade_date)]

# Merge the filtered login data with the trades data
merged_df = pd.merge(trades_dataframe, filtered_login_dataframe, on='login', how='inner')

# Combine Daily Charts

In [58]:
daily_charts = glob.glob("Dataset/daily_chart/*.csv")
all_charts = pd.DataFrame()

for chart in daily_charts:
    name = Path(chart).stem
    csv = pd.read_csv(chart, index_col='date', parse_dates=True)
    all_charts[name] = csv['close']

Convert all rates to be from AccountCurrency to USD and rename columns


In [59]:
usd_columns = [col for col in all_charts.columns if col.startswith('USD')]
all_charts_USD = all_charts.copy()
all_charts_USD[usd_columns] = 1 / all_charts_USD[usd_columns]
all_charts_USD.columns = [col.replace('USD', '') for col in all_charts_USD.columns]
all_charts_USD['USD'] = float(1)
all_charts_USD

,AUD,EUR,GBP,NZD,CAD,CHF,CNH,HKD,HUF,JPY,MXN,NOK,PLN,SEK,SGD,THB,TRY,ZAR,USD
date,,,,,,,,,,,,,,,,,,,
2023-01-03,0.67233,1.05472,1.19663,0.62510,0.732032,1.068810,0.144454,0.128003,0.002637,0.007633,0.051531,0.099209,0.225695,0.094656,0.743572,0.029099,0.053501,0.058799,1.0
2023-01-04,0.68296,1.06030,1.20527,0.62923,0.741988,1.076403,0.144975,0.127946,0.002682,0.007541,0.051634,0.099242,0.227394,0.095211,0.746798,0.029495,0.053402,0.059273,1.0
2023-01-05,0.67513,1.05207,1.18981,0.62292,0.737039,1.068079,0.145178,0.127996,0.002658,0.007496,0.051754,0.097625,0.224621,0.093806,0.743378,0.029386,0.053327,0.058268,1.0
2023-01-06,0.68674,1.06437,1.20924,0.63485,0.743859,1.079319,0.146431,0.128114,0.002702,0.007573,0.052284,0.100017,0.227017,0.095164,0.749299,0.029684,0.053385,0.058519,1.0
2023-01-09,0.69091,1.07288,1.21725,0.63708,0.747010,1.085882,0.147518,0.128157,0.002709,0.007583,0.052259,0.100845,0.228689,0.096313,0.751704,0.029948,0.053372,0.059013,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23,0.65621,1.08190,1.26710,0.61912,0.740960,1.134984,0.138903,0.127915,0.002827,0.006644,0.058499,0.094905,NaN,0.096900,0.745773,0.027824,0.032938,0.051877,1.0
2024-02-26,0.65397,1.08503,1.26843,0.61720,0.740505,1.136454,0.138720,0.127817,0.002789,0.006636,0.058492,0.095128,NaN,0.097266,0.743871,0.027873,0.032178,0.051857,1.0
2024-02-27,0.65426,1.08435,1.26837,0.61698,0.739262,1.138265,0.138654,0.127813,0.002775,0.006644,0.058609,0.094991,NaN,0.097055,0.744120,0.027882,0.032107,0.052390,1.0


# Converting Account Currency to USD

In [60]:
login_trades_rates = merged_df
login_trades_rates['trade_date'] = pd.to_datetime(pd.to_datetime(login_trades_rates['open_time']).dt.date)

all_charts_USD_stacked = all_charts_USD.stack()
login_trades_rates = pd.merge(login_trades_rates, all_charts_USD_stacked.rename('usd_rate'), how='left', left_on=['trade_date', 'account_currency'], right_index=True)

conversion_columns = ['commission', 'swaps', 'profit']
login_trades_USD = login_trades_rates

for column in conversion_columns:
    login_trades_USD[column] = login_trades_USD [column] * login_trades_USD['usd_rate']

login_trades_USD

,ticket,login,symbol,cmd,volume,open_time,open_price,close_time,close_price,tp,...,reason,commission,swaps,profit,volume_usd,country,account_currency,reg_date,trade_date,usd_rate
0,68880703,7062462,XAUUSD,0,0.01,2024-02-13 17:05:41,1991.35000,2024-02-15 16:05:14,2003.01000,0.00000,...,1,0.000000,-1.199330,11.597089,3994.36,Latvia,EUR,2023-12-22 18:51:59,2024-02-13,1.07083
1,68880910,7062462,XAUUSD,0,0.02,2024-02-13 17:08:49,1990.30000,2024-02-15 16:05:10,2002.76000,0.00000,...,1,0.000000,-2.409368,24.800423,7986.12,Latvia,EUR,2023-12-22 18:51:59,2024-02-13,1.07083
2,68120690,813125,US2000,0,1.00,2024-01-24 07:56:45,1983.22000,2024-01-24 16:41:43,1996.68000,2012.34000,...,1,0.000000,0.000000,134.137462,39799.00,United Kingdom,GBP,2023-05-20 18:43:18,2024-01-24,1.27241
3,68169249,813125,US2000,0,2.00,2024-01-24 23:36:11,1960.62000,2024-01-25 16:29:03,1991.51000,2331.85000,...,1,0.000000,-9.174076,617.449677,79042.60,United Kingdom,GBP,2023-05-20 18:43:18,2024-01-24,1.27241
4,68186877,88945036,NZDUSD,0,0.13,2024-01-25 08:00:00,0.61062,2024-01-25 15:38:26,0.61317,0.61312,...,1,-0.910000,0.000000,33.150000,15909.27,Nigeria,USD,2023-05-01 04:36:26,2024-01-25,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124162,68795989,810950,XAUUSD,0,0.10,2024-02-12 11:53:03,2020.74000,2024-02-13 07:53:25,2021.51000,2026.00000,...,5,0.000000,0.000000,7.700000,40422.50,United Kingdom,USD,2023-04-14 19:29:33,2024-02-12,1.00000
3124163,68796026,810950,XAUUSD,0,0.10,2024-02-12 11:53:14,2020.70000,2024-02-13 07:53:29,2021.50000,2026.00000,...,5,0.000000,0.000000,8.000000,40422.00,United Kingdom,USD,2023-04-14 19:29:33,2024-02-12,1.00000
3124164,68854039,810799,EURUSD,1,0.10,2024-02-13 11:56:54,1.07654,2024-02-13 12:00:16,1.07693,0.00000,...,5,-0.642498,0.000000,-3.876405,21534.70,France,EUR,2023-04-12 14:29:35,2024-02-13,1.07083
3124165,68785760,810710,GER30,1,0.03,2024-02-12 09:09:05,16994.30000,2024-02-13 11:41:03,16934.00000,0.00000,...,5,0.000000,0.140026,19.485101,10931.50,France,EUR,2023-04-11 18:29:39,2024-02-12,1.07712


In [61]:
merged_df = login_trades_USD

## Feature engineering and merging

TODO: justify feature engineering

In [62]:
def infer_tp_sl_hit(row):
    if row['cmd'] == 0:  # Buy trade
        if row['close_price'] >= row['tp'] and row['tp'] > 0:
            return 'tp_hit'
        elif row['close_price'] <= row['sl'] and row['sl'] > 0:
            return 'sl_hit'
    elif row['cmd'] == 1:  # Sell trade
        if row['close_price'] <= row['tp'] and row['tp'] > 0:
            return 'tp_hit'
        elif row['close_price'] >= row['sl'] and row['sl'] > 0:
            return 'sl_hit'
    return 'none'

In [63]:
# Count the number of trades per account
total_trades_per_login = merged_df.groupby('login')['ticket'].count()

# Count the number and percentage of buy trades per account
buy_trades_per_login = merged_df[merged_df['cmd'] == 0].groupby('login')['ticket'].count()
percentage_buys = (buy_trades_per_login / total_trades_per_login * 100).fillna(0)

# Calculate various mean averages
average_volume_per_login = merged_df.groupby('login')['volume'].mean() 
average_volume_usd_per_login = merged_df.groupby('login')['volume_usd'].mean()
average_commission_per_login = merged_df.groupby('login')['commission'].mean()
average_swaps_per_login = merged_df.groupby('login')['swaps'].mean()
average_profit_per_login = merged_df.groupby('login')['profit'].mean()

# Calculate ratio of profitable trades
profitable_trades = merged_df[merged_df['profit'] > 0]
ratio_profitable_trades = profitable_trades.groupby('login').size() / total_trades_per_login.replace(0, pd.NA)

# Calculate profit and loss variability per account
profit_loss_variability = merged_df.groupby('login')['profit'].std()

# Calculate average trade duration per account
merged_df['trade_duration'] = (merged_df['close_time'] - merged_df['open_time']).dt.total_seconds()
average_trade_duration = merged_df.groupby('login')['trade_duration'].mean()

# Calculate average DPM per account
merged_df['DPM'] = merged_df['profit'] / (merged_df['volume_usd'] / 1e6)  # Converting volume from USD to million USD
average_dpm_per_login = merged_df.groupby('login')['DPM'].mean()

# Find the most common reason per account
reason_per_login = merged_df.groupby('login')['reason'].apply(lambda x: x.value_counts().idxmax())


# Average unique symbols traded calc
# Extract the year and month from the 'open_time' for grouping
merged_df['year_month'] = merged_df['open_time'].dt.to_period('M')

# Group by 'login' and 'year_month', then calculate the unique symbols traded in each month per account
monthly_unique_symbols = merged_df.groupby(['login', 'year_month'])['symbol'].nunique().reset_index(name='unique_symbols')

# Now calculate the average number of unique symbols traded per month across all accounts
unique_symbols_traded = monthly_unique_symbols.groupby('login')['unique_symbols'].mean()



# Peak Trading Times: Most frequent trading hour per account
merged_df['trade_hour'] = merged_df['open_time'].dt.hour
peak_trading_times = merged_df.groupby('login')['trade_hour'].agg(lambda x: x.value_counts().idxmax())


# TP/SL calculations
# Add column to dataset whether tp or sl has been hit
merged_df['tp_sl_hit'] = merged_df.apply(infer_tp_sl_hit, axis=1)

# Calculate the TP and SL hit frequencies for each account
tp_hits = merged_df[merged_df['tp_sl_hit'] == 'tp_hit'].groupby('login').size()
sl_hits = merged_df[merged_df['tp_sl_hit'] == 'sl_hit'].groupby('login').size()

# Calculate the TP/SL hit frequency ratio (ensure no division by zero)
# Replace 0 with a small number to avoid division by zero or use np.where to handle 0 cases
tp_sl_hit_frequency_ratio = (tp_hits / sl_hits.replace(0, 1)).fillna(0)

# Calculate the average profit for trades where TP was hit
average_profit_tp = merged_df[merged_df['tp_sl_hit'] == 'tp_hit'].groupby('login')['profit'].mean()

# Calculate the average loss for trades where SL was hit
average_loss_sl = merged_df[merged_df['tp_sl_hit'] == 'sl_hit'].groupby('login')['profit'].mean()

# Calculate the Reward-to-Risk Ratio
# Note: Ensure no division by zero
reward_to_risk_ratio = (average_profit_tp / -average_loss_sl.replace(0, pd.NA)).fillna(0)


# Compile these metrics into a single dataframe
result_dataframe = pd.DataFrame({
    'Total_Trades': total_trades_per_login,
    'Buy_Percentage': percentage_buys,
    'Average_Volume': average_volume_per_login,
    'Average_Volume_USD': average_volume_usd_per_login,
    'Average_DPM': average_dpm_per_login,
    'Unique_Symbols_Traded': unique_symbols_traded,
    'Peak_Trading_Times': peak_trading_times,
    'Ratio_Profitable_Trades': ratio_profitable_trades,
    'Profit_Loss_Variability': profit_loss_variability,
    'Average_Trade_Duration': average_trade_duration,
    'TP/SL Hit Ratio': tp_sl_hit_frequency_ratio,
    'Reward_Risk_Ratio': reward_to_risk_ratio,
    'Most_Common_Trading_Method': reason_per_login,
    'Average_Commission': average_commission_per_login,
    'Average_Swaps': average_swaps_per_login,
    'Average_Profit': average_profit_per_login
})

C:\Users\Joel Larsen\AppData\Local\Temp\ipykernel_33336\2855158977.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  reward_to_risk_ratio = (average_profit_tp / -average_loss_sl.replace(0, pd.NA)).fillna(0)


In [64]:
final_dataset = pd.merge(login_dataframe, result_dataframe, on='login', how='inner')

In [65]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8167 entries, 0 to 8166
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   login                       8167 non-null   int64         
 1   country                     8167 non-null   object        
 2   account_currency            8167 non-null   object        
 3   reg_date                    8167 non-null   datetime64[ns]
 4   Total_Trades                8167 non-null   int64         
 5   Buy_Percentage              8167 non-null   float64       
 6   Average_Volume              8167 non-null   float64       
 7   Average_Volume_USD          8167 non-null   float64       
 8   Average_DPM                 8167 non-null   float64       
 9   Unique_Symbols_Traded       8167 non-null   float64       
 10  Peak_Trading_Times          8167 non-null   int32         
 11  Ratio_Profitable_Trades     7373 non-null   float64     

In [66]:
final_dataset.head()

,login,country,account_currency,reg_date,Total_Trades,Buy_Percentage,Average_Volume,Average_Volume_USD,Average_DPM,Unique_Symbols_Traded,Peak_Trading_Times,Ratio_Profitable_Trades,Profit_Loss_Variability,Average_Trade_Duration,TP/SL Hit Ratio,Reward_Risk_Ratio,Most_Common_Trading_Method,Average_Commission,Average_Swaps,Average_Profit
0,524974,Switzerland,USD,2023-05-17 03:13:02,143,48.251748,0.046364,1.816077e+04,96.362040,1.000000,17,0.979021,1.805020,4123.216783,0.000000,0.000000,1,0.000000,-0.022238,1.667203
1,524978,Austria,EUR,2023-06-07 05:58:36,1392,47.485632,1.230632,4.044965e+08,-3.415139,9.333333,17,0.762931,316.119097,36404.811782,3.375000,0.086763,1,-6.935588,-1.481077,25.668214
2,524979,France,USD,2023-06-07 06:17:30,2194,49.635369,0.013943,6.724644e+03,-264.410462,3.500000,17,0.718323,11.676818,57056.876937,0.087081,-0.549598,5,0.000000,-0.111285,-0.837867
3,524984,Singapore,USD,2023-10-31 08:34:08,244,38.934426,0.104262,1.836608e+06,-53.917476,3.000000,16,0.606557,42.988956,20359.959016,1.923077,-0.155076,1,0.000000,-0.073893,-0.697910
4,760487,Singapore,SGD,2023-01-04 08:48:24,69,15.942029,0.012609,2.783882e+03,-3403.656636,1.000000,17,0.463768,12.396847,46984.231884,0.000000,0.000000,1,0.000000,0.002172,-10.232797


## Longevity calculations and merging

Longevity is defined as close time of last trade minus open time of first trade for a given account (definition confirmed by Carol). 

For longevity, it's also important to consider whether an account is actively trading or is inactive. An account is considered 'active' if it trades within the mosty recent month of the dataset, and inactive if it doesn't. Accounts that have registered in the most recent month of the dataset have been removed to avoid confusion.

We do not want to label an account that becomes inactive after 1 month the same way as an account that has actively traded for 1 month, but is still ongoing. We can't define longevity for these 'active' accounts as we do not know when they will stop trading. Therefore, it stands to reason that due to having a known duration, 'inactive' accounts are better indicators of longevity, at least in the short term.

Because of this, the dataset is binned into 5 categories, where only 'inactive' accounts are included up until 360 days (trades.csv has a range of 422 days). For '360+' both active and inactive are included as this bin is open ended.

In [67]:
# Calculate the first open time and last close time for each account
first_open_time_per_login = merged_df.groupby('login')['open_time'].min()
last_close_time_per_login = merged_df.groupby('login')['close_time'].max()

# Calculate longevity as the difference in days between the last close time and first open time
longevity_per_login = (last_close_time_per_login - first_open_time_per_login).dt.days

# Define the most recent month based on the latest trade in the dataset
most_recent_month_start = merged_df['close_time'].max().replace(day=1)

# Determine active accounts (trading in the most recent month)
active_accounts = merged_df[merged_df['close_time'] >= most_recent_month_start].groupby('login').size().index

# Mark accounts as active or inactive
longevity_per_login = longevity_per_login.to_frame(name='longevity')
longevity_per_login['active'] = longevity_per_login.index.isin(active_accounts)

# Exclude accounts that were registered in the most recent month
valid_accounts = login_dataframe[login_dataframe['reg_date'] < most_recent_month_start]

# Bin the longevity values
bins = [-1, 30, 90, 180, 270, 360, float('inf')]
longevity_per_login['longevity_bin'] = pd.cut(longevity_per_login['longevity'], bins=bins, labels=False)

# Exclude active accounts from all bins except for 360+
longevity_per_login = longevity_per_login[(longevity_per_login['active'] == False) | (longevity_per_login['longevity_bin'] == 5)]

In [68]:
# Merge longevity information with result_dataframe
result_dataframe = final_dataset.reset_index().merge(longevity_per_login, on='login', how='inner')

In [69]:
# Calculate trading frequency based on total number of trades and longevity
result_dataframe['Trading_Frequency'] = result_dataframe['Total_Trades'] / result_dataframe['longevity'].replace(0, 1)

In [70]:
result_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5725 entries, 0 to 5724
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   index                       5725 non-null   int64         
 1   login                       5725 non-null   int64         
 2   country                     5725 non-null   object        
 3   account_currency            5725 non-null   object        
 4   reg_date                    5725 non-null   datetime64[ns]
 5   Total_Trades                5725 non-null   int64         
 6   Buy_Percentage              5725 non-null   float64       
 7   Average_Volume              5725 non-null   float64       
 8   Average_Volume_USD          5725 non-null   float64       
 9   Average_DPM                 5725 non-null   float64       
 10  Unique_Symbols_Traded       5725 non-null   float64       
 11  Peak_Trading_Times          5725 non-null   int32       

Filtering accounts in the ways described above has reduced our unique accounts from 8167 to 5724

In [71]:
result_dataframe.head()

,index,login,country,account_currency,reg_date,Total_Trades,Buy_Percentage,Average_Volume,Average_Volume_USD,Average_DPM,...,TP/SL Hit Ratio,Reward_Risk_Ratio,Most_Common_Trading_Method,Average_Commission,Average_Swaps,Average_Profit,longevity,active,longevity_bin,Trading_Frequency
0,1,524978,Austria,EUR,2023-06-07 05:58:36,1392,47.485632,1.230632,4.044965e+08,-3.415139,...,3.375000,0.086763,1,-6.935588,-1.481077,25.668214,142,False,2,9.802817
1,2,524979,France,USD,2023-06-07 06:17:30,2194,49.635369,0.013943,6.724644e+03,-264.410462,...,0.087081,-0.549598,5,0.000000,-0.111285,-0.837867,107,False,2,20.504673
2,4,760487,Singapore,SGD,2023-01-04 08:48:24,69,15.942029,0.012609,2.783882e+03,-3403.656636,...,0.000000,0.000000,1,0.000000,0.002172,-10.232797,13,False,0,5.307692
3,5,804664,Malaysia,USD,2023-09-24 07:02:49,85,63.529412,0.018706,5.490400e+03,17.604088,...,NaN,NaN,5,0.000000,-0.023176,-0.524588,9,False,0,9.444444
4,6,804687,Australia,AUD,2023-01-10 10:26:00,484,51.239669,0.019773,5.271755e+05,-45.416080,...,0.002283,55.989328,5,-0.101129,-0.009749,-0.210692,372,False,5,1.301075


In [72]:
result_dataframe['longevity_bin'].value_counts()

longevity_bin
0    3575
1    1216
2     595
3     214
5      69
4      56
Name: count, dtype: int64

Distribution of longevity. Be aware that 5 appears before 4, because there are more accounts in that bin (most likely due to including both active and inactive accounts)

In [73]:
# Filter rows where 'longevity_bin' is 5
filtered_df = result_dataframe[result_dataframe['longevity_bin'] == 5]

# Count the occurrences of True and False in the 'active' column for the filtered rows
active_counts = filtered_df['active'].value_counts()

# Display the counts
print(active_counts)

active
True     63
False     6
Name: count, dtype: int64


Distribution of '360+' days by activity status.

## Merge result_dataframe with daily_report

In [74]:
# Calculate daily average net deposit and binary check for credit per account
account_averages = daily_reports_df.groupby('login').agg(
    average_net_deposit=('net_deposit', 'mean'),  # Overall average net deposit per account
    has_credit=('credit', lambda x: 1 if any(y > 0 for y in x) else 0)  # Overall binary check for credit
).reset_index()

# Calculate net_deposit frequency ratio for each login across all entries
deposit_frequency_data = daily_reports_df.groupby('login')['net_deposit'].apply(
    lambda x: (x != 0).sum() / (x == 0).sum() if (x == 0).sum() != 0 else (x != 0).sum()
).reset_index(name='net_deposit_frequency_ratio')

account_averages = account_averages.merge(deposit_frequency_data, on='login', how='left')

In [75]:
result_dataframe = result_dataframe.merge(account_averages, on='login', how='left')

In [76]:
result_dataframe.head()

,index,login,country,account_currency,reg_date,Total_Trades,Buy_Percentage,Average_Volume,Average_Volume_USD,Average_DPM,...,Average_Commission,Average_Swaps,Average_Profit,longevity,active,longevity_bin,Trading_Frequency,average_net_deposit,has_credit,net_deposit_frequency_ratio
0,1,524978,Austria,EUR,2023-06-07 05:58:36,1392,47.485632,1.230632,4.044965e+08,-3.415139,...,-6.935588,-1.481077,25.668214,142,False,2,9.802817,-148.587567,0.0,0.056225
1,2,524979,France,USD,2023-06-07 06:17:30,2194,49.635369,0.013943,6.724644e+03,-264.410462,...,0.000000,-0.111285,-0.837867,107,False,2,20.504673,6.950570,0.0,0.047809
2,4,760487,Singapore,SGD,2023-01-04 08:48:24,69,15.942029,0.012609,2.783882e+03,-3403.656636,...,0.000000,0.002172,-10.232797,13,False,0,5.307692,2.490144,0.0,0.004819
3,5,804664,Malaysia,USD,2023-09-24 07:02:49,85,63.529412,0.018706,5.490400e+03,17.604088,...,0.000000,-0.023176,-0.524588,9,False,0,9.444444,0.167482,0.0,0.007246
4,6,804687,Australia,AUD,2023-01-10 10:26:00,484,51.239669,0.019773,5.271755e+05,-45.416080,...,-0.101129,-0.009749,-0.210692,372,False,5,1.301075,2.798054,0.0,0.022388


In [77]:
result_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5725 entries, 0 to 5724
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   index                        5725 non-null   int64         
 1   login                        5725 non-null   int64         
 2   country                      5725 non-null   object        
 3   account_currency             5725 non-null   object        
 4   reg_date                     5725 non-null   datetime64[ns]
 5   Total_Trades                 5725 non-null   int64         
 6   Buy_Percentage               5725 non-null   float64       
 7   Average_Volume               5725 non-null   float64       
 8   Average_Volume_USD           5725 non-null   float64       
 9   Average_DPM                  5725 non-null   float64       
 10  Unique_Symbols_Traded        5725 non-null   float64       
 11  Peak_Trading_Times           5725 non-null 

In [78]:
result_dataframe['has_credit'].value_counts()

has_credit
0.0    4247
1.0    1477
Name: count, dtype: int64

In [79]:
result_dataframe['net_deposit_frequency_ratio'].describe()

count    5724.000000
mean        0.030173
std         0.057927
min         0.000000
25%         0.010870
50%         0.020270
75%         0.032967
max         1.500000
Name: net_deposit_frequency_ratio, dtype: float64

## Final pre-processing

Simple pre-processing including converting less important trading methods to 'Other' and then reverse encoding, filling NaN values, and reording columns.

In [80]:
result_dataframe['Most_Common_Trading_Method'].value_counts()

Most_Common_Trading_Method
5    2527
0    1957
1    1088
6     142
3      10
4       1
Name: count, dtype: int64

In [81]:
# Filter out the dataset
result_dataframe['Most_Common_Trading_Method'] = result_dataframe['Most_Common_Trading_Method'].apply(lambda x: x if x in [0, 1, 5] else 7)

# Rename the codes to strings
result_dataframe['Trading_Method'] = result_dataframe['Most_Common_Trading_Method'].map({0: 'Client', 1: 'Expert', 5: 'Mobile', 7:'Other'})
result_dataframe.drop('Most_Common_Trading_Method', axis=1, inplace=True)

In [82]:
# Remove reg_date
# Remove Total_Trades
# Move trading frequency to front, and active to back
result_dataframe.drop(['index', 'reg_date'], axis=1, inplace=True)

In [83]:
nan_counts = result_dataframe.isna().sum()
print("Count of NaN values per column:")
print(nan_counts)

Count of NaN values per column:
login                             0
country                           0
account_currency                  0
Total_Trades                      0
Buy_Percentage                    0
Average_Volume                    0
Average_Volume_USD                0
Average_DPM                       0
Unique_Symbols_Traded             0
Peak_Trading_Times                0
Ratio_Profitable_Trades         756
Profit_Loss_Variability         556
Average_Trade_Duration            0
TP/SL Hit Ratio                1970
Reward_Risk_Ratio              1970
Average_Commission                0
Average_Swaps                     0
Average_Profit                    0
longevity                         0
active                            0
longevity_bin                     0
Trading_Frequency                 0
average_net_deposit               1
has_credit                        1
net_deposit_frequency_ratio       1
Trading_Method                    0
dtype: int64


In [84]:
result_dataframe.fillna(0, inplace=True)

In [85]:
result_dataframe['country'] = result_dataframe['country'].astype('category')
result_dataframe['account_currency'] = result_dataframe['account_currency'].astype('category')
result_dataframe['Trading_Method'] = result_dataframe['Trading_Method'].astype('category')
result_dataframe['Peak_Trading_Times'] = result_dataframe['Peak_Trading_Times'].astype('category')
result_dataframe['has_credit'] = result_dataframe['has_credit'].astype('category')

In [86]:
cols = list(result_dataframe.columns)

cols.remove('Trading_Frequency')
cols.remove('active')

# Insert 'Trading Frequency' at the new position
cols.insert(3, 'Trading_Frequency')

# Insert 'Active' at the new position
cols.insert(23, 'active')

cols = [col for col in cols if col not in ['longevity', 'longevity_bin']]
cols += ['longevity', 'longevity_bin']

# Reorder the DataFrame based on the new column list
final_dataset = result_dataframe[cols]

In [87]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5725 entries, 0 to 5724
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   login                        5725 non-null   int64   
 1   country                      5725 non-null   category
 2   account_currency             5725 non-null   category
 3   Trading_Frequency            5725 non-null   float64 
 4   Total_Trades                 5725 non-null   int64   
 5   Buy_Percentage               5725 non-null   float64 
 6   Average_Volume               5725 non-null   float64 
 7   Average_Volume_USD           5725 non-null   float64 
 8   Average_DPM                  5725 non-null   float64 
 9   Unique_Symbols_Traded        5725 non-null   float64 
 10  Peak_Trading_Times           5725 non-null   category
 11  Ratio_Profitable_Trades      5725 non-null   float64 
 12  Profit_Loss_Variability      5725 non-null   float64 
 13  Ave

In [88]:
final_dataset.describe()

,login,Trading_Frequency,Total_Trades,Buy_Percentage,Average_Volume,Average_Volume_USD,Average_DPM,Unique_Symbols_Traded,Ratio_Profitable_Trades,Profit_Loss_Variability,Average_Trade_Duration,TP/SL Hit Ratio,Reward_Risk_Ratio,Average_Commission,Average_Swaps,Average_Profit,average_net_deposit,net_deposit_frequency_ratio,longevity,longevity_bin
count,5.725000e+03,5725.000000,5725.000000,5725.000000,5725.000000,5.725000e+03,5725.000000,5725.000000,5725.000000,5725.000000,5.725000e+03,5725.000000,5725.000000,5725.000000,5725.000000,5725.000000,5725.000000,5725.000000,5725.000000,5725.000000
mean,4.260059e+06,24.677048,242.507424,44.600729,0.386369,2.283864e+08,-145.503040,3.727612,0.472580,159.869197,6.628252e+04,1.763087,1.714086,-0.410879,0.184834,-9.806129,5.303990,0.030168,45.232664,0.631790
std,1.654573e+07,710.948151,1023.377574,28.085715,1.621218,1.838682e+09,3692.263888,4.109689,0.283212,1696.792091,2.094645e+05,27.804664,223.521743,3.629525,31.930857,720.966109,73.908944,0.057923,70.013825,1.011621
min,5.249780e+05,0.010178,1.000000,0.000000,0.005000,1.040000e+00,-109247.458671,1.000000,0.000000,0.000000,0.000000e+00,0.000000,-8311.305640,-108.598893,-1173.062580,-38585.491238,-2474.894189,0.000000,0.000000,0.000000
25%,8.148130e+05,1.000000,8.000000,26.402189,0.028333,2.613592e+04,-467.261699,1.000000,0.285714,5.781964,4.441059e+03,0.000000,0.000000,0.000000,-0.060256,-16.592206,0.015000,0.010870,2.000000,0.000000
50%,8.215830e+05,2.486111,33.000000,46.650718,0.078333,3.663738e+05,-71.891849,2.000000,0.500000,23.995345,1.570000e+04,0.000000,0.000000,0.000000,0.000000,-1.781250,1.182138,0.020270,15.000000,0.000000
75%,7.056823e+06,7.000000,142.000000,60.000000,0.270000,2.200197e+07,44.698309,4.750000,0.666667,83.279900,5.841114e+04,0.090909,0.199083,0.000000,0.000486,0.220833,4.354294,0.032967,60.000000,1.000000
max,1.000055e+09,46509.000000,46509.000000,100.000000,39.285714,6.827463e+10,112017.955519,31.500000,1.000000,87148.616792,6.793694e+06,1245.000000,9557.708333,0.000000,1045.468282,15005.000000,2902.991884,1.500000,415.000000,5.000000


In [89]:
for col in final_dataset.columns:
    # Check if the column is of type 'category'
    if final_dataset[col].dtype.name == 'category':
        # Display the value counts for categorical columns
        print("Value Counts for", col, ":")
        print(final_dataset[col].value_counts())
        print()

Value Counts for country :
country
Indonesia                1209
Vietnam                   761
United Kingdom            474
France                    469
Canada                    421
                         ... 
Myanmar                     1
Republic of the Congo       1
Reunion                     1
Russia                      1
Zimbabwe                    1
Name: count, Length: 140, dtype: int64

Value Counts for account_currency :
account_currency
USD    3901
EUR     983
GBP     425
CAD     207
AUD      80
NZD      67
SGD      54
CHF       8
Name: count, dtype: int64

Value Counts for Peak_Trading_Times :
Peak_Trading_Times
16    888
15    819
17    722
10    459
11    317
9     276
18    257
14    238
12    238
13    183
20    153
8     152
19    150
4     135
5     130
21    102
22     86
7      85
3      80
6      73
1      57
23     49
2      44
0      32
Name: count, dtype: int64

Value Counts for has_credit :
has_credit
0.0    4248
1.0    1477
Name: count, dtype: int64

Val

In [90]:
final_dataset

,login,country,account_currency,Trading_Frequency,Total_Trades,Buy_Percentage,Average_Volume,Average_Volume_USD,Average_DPM,Unique_Symbols_Traded,...,Average_Commission,Average_Swaps,Average_Profit,average_net_deposit,has_credit,active,net_deposit_frequency_ratio,Trading_Method,longevity,longevity_bin
0,524978,Austria,EUR,9.802817,1392,47.485632,1.230632,4.044965e+08,-3.415139,9.333333,...,-6.935588,-1.481077,25.668214,-148.587567,0.0,False,0.056225,Expert,142,2
1,524979,France,USD,20.504673,2194,49.635369,0.013943,6.724644e+03,-264.410462,3.500000,...,0.000000,-0.111285,-0.837867,6.950570,0.0,False,0.047809,Mobile,107,2
2,760487,Singapore,SGD,5.307692,69,15.942029,0.012609,2.783882e+03,-3403.656636,1.000000,...,0.000000,0.002172,-10.232797,2.490144,0.0,False,0.004819,Expert,13,0
3,804664,Malaysia,USD,9.444444,85,63.529412,0.018706,5.490400e+03,17.604088,6.000000,...,0.000000,-0.023176,-0.524588,0.167482,0.0,False,0.007246,Mobile,9,0
4,804687,Australia,AUD,1.301075,484,51.239669,0.019773,5.271755e+05,-45.416080,2.769231,...,-0.101129,-0.009749,-0.210692,2.798054,0.0,False,0.022388,Mobile,372,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5720,88944953,Canada,CAD,0.141732,54,37.037037,0.041667,1.621176e+07,-24.464597,3.333333,...,0.000000,-0.005931,-0.134167,0.122249,0.0,True,0.004914,Mobile,381,5
5721,88944971,Malaysia,USD,4.420000,442,46.153846,1.389367,1.249156e+09,-298.590702,7.000000,...,0.000000,-5.085113,-51.760588,1.870890,0.0,False,0.002625,Expert,100,2
5722,88945034,Singapore,USD,10.413043,479,40.083507,0.043278,1.522641e+07,-70.501778,11.333333,...,-0.256347,-0.000271,-1.519937,0.000000,0.0,False,0.000000,Expert,46,1
5723,88945038,Singapore,SGD,5.162055,1306,58.805513,0.078913,1.824879e+04,140.220508,4.111111,...,-0.700736,-0.125105,-0.125379,0.000000,0.0,False,0.000000,Expert,253,3


In [91]:
# Remove outliers based on multiple conditions
cleaned_dataset = final_dataset[
    (final_dataset['Trading_Frequency'] <= 1000) &
    (final_dataset['Profit_Loss_Variability'] <= 10000) &
    (final_dataset['Average_Trade_Duration'] <= 3000000) &
    (final_dataset['TP/SL Hit Ratio'] <= 400) &
    ((final_dataset['Reward_Risk_Ratio'] >= -5000) & (final_dataset['Reward_Risk_Ratio'] <= 5000)) &
    (final_dataset['Average_Profit'] >= -20000) &
    ((final_dataset['average_net_deposit'] >= -2000) & (final_dataset['average_net_deposit'] <= 2000)) &
    (final_dataset['net_deposit_frequency_ratio'] <= 1)
]

# Display the shape of the dataset before and after removing outliers
print("Original Dataset Size:", final_dataset.shape)
print("Cleaned Dataset Size:", cleaned_dataset.shape)

Original Dataset Size: (5725, 26)
Cleaned Dataset Size: (5692, 26)


## Export

In [47]:
cleaned_dataset.to_csv('Dataset/output_dataset.csv', index=False)